In [ ]:
# for remote notebooks, colab etc.
!wget https://raw.githubusercontent.com/xenova/transformers.js/main/scripts/requirements.txt
!wget https://raw.githubusercontent.com/xenova/transformers.js/main/scripts/convert.py

In [ ]:
!pip install -r requirements.txt -q
!pip install huggingface_hub -q

In [3]:
from huggingface_hub import HfApi, notebook_login, whoami, create_repo, ModelCard, ModelCardData

**Log in to 🤗 hub**

In [ ]:
# uncomment when running in vscode
# !pip install ipywidgets -q
notebook_login()

In [ ]:
user = whoami()['name']
api = HfApi()
print(f"Logged in to 🤗 as {user}")

**Model to convert**  
Pick a model from the  [🤗 Hub](https://huggingface.co/models).  

In [5]:
model = "stabilityai/stablelm-2-1_6b" # @param {type:"string"}
user = "" # @param {type:"string"}

**Convert the model to ONNX**
This might take a moment.

In [7]:
# check availabile arguments
!python convert.py --help

/Users/davidrechkemmer/projects/javascript/transformers.js/.venv/lib/python3.11/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
usage: convert.py [-h] --model_id MODEL_ID [--tokenizer_id TOKENIZER_ID]
                  [--quantize [QUANTIZE]]
                  [--output_parent_dir OUTPUT_PARENT_DIR] [--task TASK]
                  [--opset OPSET] [--device DEVICE]
                  [--skip_validation [SKIP_VALIDATION]]
                  [--per_channel [PER_CHANNEL]]
                  [--reduce_range [REDUCE_RANGE]]
                  [--output_attentions [OUTPUT_ATTENTIONS]]
                  [--split_modalities [SPLIT_MODALITIES]]
                  [--trust_remote_code [TRUST_REMOTE_CODE]]
                  [--custom_onnx_configs CUSTOM_ONNX_CONFIGS]

options:
  -h, --help            show this help message and e

In [ ]:
!python convert.py --quantize --model_id $model

In [ ]:
model_name = model.split("/")[-1]
model_org = user or model.split("/")[0] # if user is not provided, use the model org
repo_id = f"{model_org}/{model_name}" # new repo id
print(f"Repo ID: {repo_id}")

**Create new repo** 

In [ ]:
repo_url = create_repo(repo_id, exist_ok=True) # creates a new repo

In [ ]:
# upload the model to the hub
api.upload_folder(
    folder_path=f"models/{model}", # default output path from convert.py
    repo_id=repo_id,
)

In [ ]:
# modify to your needs
card = ModelCard.load(model) # get the old model card
card_meta = card.data.to_dict()
card_meta['library_name'] = "transformers.js"
card_meta['tags'] += ["onnx"]
card_meta = ModelCardData(**card_meta)

In [ ]:
# Add transformers.js modelcard template to existing model card
# modify to your needs
content = f"""
---
{card_meta.to_yaml()}
---

https://huggingface.co/{model} with ONNX weights to be compatible with Transformers.js.  

{card.text}  
---
  
Note: Having a separate repo for ONNX weights is intended to be a temporary solution until WebML gains more traction. If you would like to make your models web-ready, we recommend converting to ONNX using [🤗 Optimum](https://huggingface.co/docs/optimum/index) and structuring your repo like this one (with ONNX weights located in a subfolder named `onnx`).
"""

In [ ]:
ModelCard(content).push_to_hub(repo_id) # push the new model card to the hub